In [1]:
from IEMOCAP_reader import IEMOCAPReader
from dataframe_processing import Relabel, Filter, LabelEncoder, OutputMerger
from feature_extractor import Wav2Vec2Embeddings, OpensmileExtractor, Spectrogram
# from splits import Split, GroupSplit, LeaveOneOut
from normalize import NormalizationStatistics
# from generators import BatchGenerator
# from dienen_model import DienenModel
from data_processors import PadDP, ToNumpyDP, SqueezeDP, NormalizeDP, LoadDataframeDP, DownsampleDP
import pandas as pd
import pickle
import numpy as np
import os

In [2]:
data = pd.read_csv('/Users/agnieszkalenart/Documents/mannheim/master_thesis/iemocap_with_features_full.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/agnieszkalenart/Documents/mannheim/master_thesis/iemocap_with_features_full.csv'

In [34]:
from IEMOCAP_reader import IEMOCAPReader
from dataframe_processing import Relabel, Filter, LabelEncoder, OutputMerger
from feature_extractor import Wav2Vec2Embeddings, OpensmileExtractor, Spectrogram
# from splits import Split, GroupSplit, LeaveOneOut
from normalize import NormalizationStatistics
# from generators import BatchGenerator
# from dienen_model import DienenModel
from data_processors import PadDP, ToNumpyDP, SqueezeDP, NormalizeDP
import pandas as pd
import pickle
import numpy as np
import ast



DATA_PATH = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/IEMOCAP'

#read data
iemocap_reader = IEMOCAPReader()
df_data = iemocap_reader.process(data_path=DATA_PATH, min_duration=0.0, min_sad_frames_duration=0, sample=None,compute_speech_rate=True)
print(df_data.loc[['Ses01F_impro04_M010']])

#create wav2vec2 features
wav2vec2_model_path = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/wav2vec2/wav2vec_small.pt'
wav2vec2_dict_path = None
normalization_axis = 1
wav2vec2_padding_axis = 1
max_size = 240000
mode  = 'sequence'
output_column = 'wav2vec2'
layer = 'output'
save_feature_files = True

wav2vec2 = Wav2Vec2Embeddings()
model, cfg, task = wav2vec2.build_wav2vec_model(model_path=wav2vec2_model_path)
data = wav2vec2.process(data = df_data,
                model_path=wav2vec2_model_path,
                 max_size=max_size,
                 mode= mode,
                 output_column=output_column,
                 layer = layer,
                 save_feature_files= save_feature_files)

100%|██████████| 1819/1819 [00:00<00:00, 3671.16it/s]
/Users/agnieszkalenart/miniconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
/Users/agnieszkalenart/miniconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


Removing 0 samples with less duration than 0.0 seconds
Removing 0 samples with less speech activity than 0 frames
                                                              filename  \
Ses01F_impro04_M010  /Users/agnieszkalenart/Documents/mannheim/mast...   

                                     wavfile modality modality_number  \
Ses01F_impro04_M010  Ses01F_impro04_M010.wav    impro               4   

                    modality_code subject  session  start   end  duration  \
Ses01F_impro04_M010       impro04  Ses01F        1      0  1.75      1.75   

                     ...       sad_times  sad_dur           words_data  \
Ses01F_impro04_M010  ...  [(0.34, 0.58)]       24  [[0.34, 0.58, HMM]]   

                     syl_avg_dur syl_avg_rate syl_times annotations_entropy  \
Ses01F_impro04_M010          NaN          NaN        []                 0.0   

                    annotations_entropy_self_report  \
Ses01F_impro04_M010                             0.0   

                

100%|██████████| 1819/1819 [14:11<00:00,  2.14it/s]


In [35]:
data['wav2vec2'] = data['wav2vec2'].apply(lambda x: [item for sublist in x for item in sublist])
data['wav2vec2'] = data['wav2vec2'].apply(np.array)
cmn_emb = dict(zip(data.index, data['wav2vec2']))


# Save the dictionary as a pickle file
with open('cmn_audio_wav2vec2.pickle', 'wb') as file:
    pickle.dump(cmn_emb, file)

In [18]:
cmn_emb['Ses01F_impro01_F000']

[array([ 3.71650457e-01,  3.10370088e-01,  6.84092864e-02,  1.88303858e-01,
         1.67854115e-01, -1.04794234e-01,  1.45410597e-01, -1.26402244e-01,
        -2.60968879e-02,  1.54478490e-01,  1.02868415e-01, -6.46802008e-01,
        -2.78459728e-01,  3.47395450e-01,  3.39937150e-01, -3.30245823e-01,
         6.27928436e-01, -6.82377741e-02,  1.28153861e-02, -2.38761008e-01,
        -2.64451623e-01, -5.21454737e-02, -2.15647793e+00,  1.57691091e-01,
         9.03737068e-01, -5.55457547e-02, -1.43639922e+00,  3.21358033e-02,
        -1.66050699e-02,  1.82940289e-01, -1.48561388e-01,  5.91020659e-02,
         2.27672651e-01, -3.43817696e-02,  3.76075983e-01,  8.69430304e-02,
        -2.91353405e-01,  5.17197624e-02, -2.93047607e-01,  4.27882373e-02,
         2.31410209e-02,  1.05407536e-02, -2.87330300e-01, -2.25516558e-01,
        -5.33549488e-03, -5.78162551e-01, -3.09167176e-01,  9.48761851e-02,
         2.08006963e-01, -3.54972988e-01, -2.35827684e-01,  2.64392138e-01,
         7.7

In [4]:
import ast

#data['wav2vec2'] = data['wav2vec2'].apply(lambda x: ast.literal_eval(x))
#data['wav2vec2'] = data['wav2vec2'].apply(lambda x: x[0])
data['opensmile'] = data['opensmile'].apply(lambda x: ast.literal_eval(x))
#data['spectrogram'] = data['spectrogram'].apply(lambda x: ast.literal_eval(x))


In [5]:
data['opensmile'] = data['opensmile'].apply(np.array)

In [6]:
data['opensmile'] = data['opensmile'].apply(lambda x: x.reshape(6373,))

In [7]:
data['opensmile'][0].shape

(6373,)

In [4]:
data[data['Unnamed: 0']=='Ses01F_impro04_M010']

,Unnamed: 0,filename,wavfile,modality,modality_number,modality_code,subject,session,start,end,...,syl_avg_dur,syl_avg_rate,syl_times,annotations_entropy,annotations_entropy_self_report,annotations_full_agreement,sentences,classID,opensmile,spectrogram


In [9]:
opensmile_dict = dict(zip(data['Unnamed: 0'], data['opensmile']))


In [10]:
# Save the dictionary as a pickle file
with open('cmn_audio_opensmile.pickle', 'wb') as file:
    pickle.dump(opensmile_dict, file)

In [54]:
list_features = ['wav2vec2', 'opensmile', 'spectrogram']
for feature in list_features:
    for i in range(5):
        print('length of ',feature,' row ', i, 'is ',len(data[feature][i]), ' x ', len(data[feature][i][0]))

length of  wav2vec2  row  0 is  97  x  768
length of  wav2vec2  row  1 is  68  x  768
length of  wav2vec2  row  2 is  156  x  768
length of  wav2vec2  row  3 is  201  x  768
length of  wav2vec2  row  4 is  137  x  768
length of  opensmile  row  0 is  1  x  6373
length of  opensmile  row  1 is  1  x  6373
length of  opensmile  row  2 is  1  x  6373
length of  opensmile  row  3 is  1  x  6373
length of  opensmile  row  4 is  1  x  6373
length of  spectrogram  row  0 is  312  x  201
length of  spectrogram  row  1 is  222  x  201
length of  spectrogram  row  2 is  501  x  201
length of  spectrogram  row  3 is  645  x  201
length of  spectrogram  row  4 is  441  x  201


In [55]:
data['wav2vec2'] = data['wav2vec2'].apply(np.array)
data['opensmile'] = data['opensmile'].apply(np.array)

In [56]:
print(data['opensmile'])

0    [[0.4211056, 0.4759358, 0.2032086, 0.1179737, ...
1    [[0.9830519, 0.4961832, 0.3816794, 0.09562548,...
2    [[0.8850114, 0.8039216, 0.0, 0.09186696, 0.104...
3    [[2.740377, 0.4126582, 0.1367089, 0.1386447, 0...
4    [[4.135432, 0.1828358, 0.9738806, 0.1455787, 0...
Name: opensmile, dtype: object


In [57]:
downsampleDP = DownsampleDP()
data = downsampleDP.process(data=data, factor = 2, col_in='opensmile',col_out='opensmile', axis = 1)

/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/ser_wav2vec_reimplement/data_processors.py:85: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  y[slc] = x


In [58]:
padDP = PadDP()
data = padDP.process(data=data, col_in = 'opensmile', col_out = 'opensmile', max_length=400)

/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/ser_wav2vec_reimplement/data_processors.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[slc] = x[slc]


In [59]:
#axis = 0?
padDP = PadDP()
data = padDP.process(data=data, col_in = 'wav2vec2', col_out = 'wav2vec2', max_length=400, axis =0)

/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/ser_wav2vec_reimplement/data_processors.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[slc] = x[slc]


In [60]:
squeezeDP = SqueezeDP()
data = squeezeDP.process(data=data, col_in = 'wav2vec2', col_out = 'wav2vec2')

In [61]:
normalization_statistics = NormalizationStatistics()
statistics = normalization_statistics.process(data=data, normalization_by='subject', column ='wav2vec2')
normalizer = NormalizeDP()
data = normalizer.process(data=data, column_in = 'wav2vec2', column_out = 'wav2vec2', statistics=statistics)

In [69]:
print(data['opensmile'])

0    [[0.4485207, 0.16059115, 0.1616628, 0.03528049...
1    [[0.73961755, 0.23865244, 0.12512985, 0.024610...
2    [[0.8444665, 0.04593348, 0.14516535, 0.0470593...
3    [[1.5765176, 0.1376768, 0.69759115, 0.37484245...
4    [[2.1591339, 0.55972965, 0.61022675, 0.3419897...
Name: opensmile, dtype: object


In [62]:
toNumpyDP = ToNumpyDP()
opensmile_data = toNumpyDP.process(data=data, col_in = 'opensmile')

In [63]:
toNumpyDP = ToNumpyDP()
wav2vec2_data = toNumpyDP.process(data=data, col_in = 'wav2vec2')

In [66]:
opensmile_data.shape

(5, 400, 3187)

In [67]:
wav2vec2_data.shape

(5, 400, 768)

In [78]:
opensmile_data[0][0][0]

0.4485207

In [16]:
import arff
import os
TMPARFFPATH = "/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/tmp/tmp.arff"


In [17]:
data = arff.load(open(TMPARFFPATH, 'r'))
print(data['data'])

[['unknown', 0.4211056, 0.4759358, 0.2032086, 0.1179737, 0.1347909, 0.1885347, 0.01681723, 0.05374375, 0.07056098, 0.09733298, 0.5063254, 0.4089924, 0.09974337, 2.021333, 6.238541, 0.22, 0.67, 0.05, 0.2548529, 0.2406417, 0.1069519, 0.06417112, 0.03743315, 0.4247312, 0.5243243, 0.0004625406, -1.484635, 0.4168662, 0.1583346, -0.1107965, 0.03218116, 1.861224, 0.3796791, 0.5775401, 0.8340849, 0.9780678, 1.160645, 0.1439829, 0.1825774, 0.3265603, 0.6233002, 2.3355, 1.712199, 0.3573072, 1.570857, 5.583004, 0.1609091, 0.54, 0.04, 0.166212, 0.3315508, 0.1176471, 0.02139037, 0.01604278, 0.516129, 0.6216216, 0.01260227, -1.358078, 0.2582861, 0.1357087, 0.002304379, -0.02961385, 0.03975631, 0.3850268, 0.2192513, 0.002503606, 0.002860716, 0.004547893, 0.0003571103, 0.001687177, 0.002044287, 0.001869933, 0.04073338, 0.03886345, 0.01064767, 2.172249, 6.193054, 0.2, 0.69, 0.04, 0.2458455, 0.1604278, 0.1229947, 0.0855615, 0.04812834, 0.4892473, 0.5081081, 9.059018e-07, -2.301733, 1.981482, -0.7958212,

In [18]:
filename = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F000.wav'

In [19]:
MODEL_PATH = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/wav2vec2/wav2vec_small_960h.pt'

CONF_PATH = "/Users/agnieszkalenart/Downloads/opensmile-3.0.0/config/is09-13/IS13_ComParE.conf"
TMPARFFPATH = "/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/tmp/tmp.arff"


In [20]:
os.system("SMILExtract -C " + CONF_PATH + " -I " + filename + " -O " + TMPARFFPATH)
data = arff.load(open(TMPARFFPATH, 'r'))
data_list = []
            #print(type(data['data']))
            #for i in range(len(data['data'])):
data_list.append((data['data'][0][1:-1]))

(MSG) [2] SMILExtract: openSMILE starting!
(MSG) [2] SMILExtract: config file is: /Users/agnieszkalenart/Downloads/opensmile-3.0.0/config/is09-13/IS13_ComParE.conf
(MSG) [2] cComponentManager: successfully registered 103 component types.
(MSG) [2] instance 'lldsink': No filename given, disabling this sink component.
(MSG) [2] instance 'lldhtksink': No filename given, disabling this sink component.
(MSG) [2] instance 'lldarffsink': No filename given, disabling this sink component.
(MSG) [2] instance 'csvsink': No filename given, disabling this sink component.
(MSG) [2] instance 'htksink': No filename given, disabling this sink component.
(MSG) [2] cComponentManager: successfully finished createInstances (48 component instances were finalised, 1 data memories were finalised)
(MSG) [2] cComponentManager: starting single thread processing loop
(WRN) [2] cComponentManager: The following component(s) could not perform any work because destination levels are full but no other component perfor

In [21]:
data_list

[[0.4211056,
  0.4759358,
  0.2032086,
  0.1179737,
  0.1347909,
  0.1885347,
  0.01681723,
  0.05374375,
  0.07056098,
  0.09733298,
  0.5063254,
  0.4089924,
  0.09974337,
  2.021333,
  6.238541,
  0.22,
  0.67,
  0.05,
  0.2548529,
  0.2406417,
  0.1069519,
  0.06417112,
  0.03743315,
  0.4247312,
  0.5243243,
  0.0004625406,
  -1.484635,
  0.4168662,
  0.1583346,
  -0.1107965,
  0.03218116,
  1.861224,
  0.3796791,
  0.5775401,
  0.8340849,
  0.9780678,
  1.160645,
  0.1439829,
  0.1825774,
  0.3265603,
  0.6233002,
  2.3355,
  1.712199,
  0.3573072,
  1.570857,
  5.583004,
  0.1609091,
  0.54,
  0.04,
  0.166212,
  0.3315508,
  0.1176471,
  0.02139037,
  0.01604278,
  0.516129,
  0.6216216,
  0.01260227,
  -1.358078,
  0.2582861,
  0.1357087,
  0.002304379,
  -0.02961385,
  0.03975631,
  0.3850268,
  0.2192513,
  0.002503606,
  0.002860716,
  0.004547893,
  0.0003571103,
  0.001687177,
  0.002044287,
  0.001869933,
  0.04073338,
  0.03886345,
  0.01064767,
  2.172249,
  6.193054,
